In [1]:
import pandas as pd
import numpy as np

In [21]:
df = pd.read_csv(
   "C:/Users/Abhi/Desktop/Python_Data_Analytics_Projects/car_insurance_pricing_analytics/data/processed/car_insurance_pricing_features.csv"
)

df.head()

,policy_id,policy_start_date,vehicle_type,policy_type,region,vehicle_age,driver_age,base_premium,discount_pct,final_premium,...,policy_year,underwriting_margin,premium_per_vehicle_age,discount_applied,discount_bucket,avg_vehicle_premium,premium_deviation_pct,risk_adjusted_discount,pricing_status,effective_price_index
0,POL10001,2023-04-13,SUV,Third Party,West,12,41,16600,0,16600.0,...,2023,8924.196385,1276.923077,0,No Discount,16035.203145,3.522231,0.000000,Profitable,1.035222
1,POL10002,2023-11-27,Sedan,Third Party,West,7,23,14100,0,14100.0,...,2023,2989.708434,1762.500000,0,No Discount,13138.717949,7.316407,0.000000,Profitable,1.073164
2,POL10003,2024-05-06,Luxury,Third Party,West,4,53,19200,0,19200.0,...,2024,7489.870605,3840.000000,0,No Discount,20650.765125,-7.025237,0.000000,Profitable,0.929748
3,POL10004,2024-07-14,Sedan,Third Party,West,11,36,13300,0,13300.0,...,2024,7497.973768,1108.333333,0,No Discount,13138.717949,1.227533,0.000000,Profitable,1.012275
4,POL10005,2024-07-16,Sedan,Comprehensive,North,2,57,13100,5,12445.0,...,2024,6618.130912,4148.333333,1,Low,13138.717949,-5.279952,1.666667,Profitable,0.947200


In [5]:
impact_summary = (
    df.groupby("discount_applied")
      .agg(
          policies=("policy_id", "count"),
          avg_premium=("final_premium", "mean"),
          avg_margin=("underwriting_margin", "mean"),
          total_margin=("underwriting_margin", "sum")
      )
      .reset_index()
)

impact_summary["discount_applied"] = impact_summary["discount_applied"].map(
    {0: "No Discount", 1: "Discounted"}
)

impact_summary


,discount_applied,policies,avg_premium,avg_margin,total_margin
0,No Discount,1309,14700.611154,5816.846218,7.614252e+06
1,Discounted,1691,13202.871082,5275.306444,8.920543e+06


In [7]:
bucket_impact = (
    df.groupby("discount_bucket")
      .agg(
          policies=("policy_id", "count"),
          avg_discount=("discount_pct", "mean"),
          avg_margin=("underwriting_margin", "mean"),
          total_margin=("underwriting_margin", "sum")
      )
      .reset_index()
)

bucket_impact


,discount_bucket,policies,avg_discount,avg_margin,total_margin
0,High,305,15.0,4752.812961,1.449608e+06
1,Low,771,5.0,5473.957701,4.220421e+06
2,Medium,615,10.0,5285.388385,3.250514e+06
3,No Discount,1309,0.0,5816.846218,7.614252e+06


In [9]:
SAFE_DISCOUNT = 10


In [13]:
high_discount_loss = df[
    df["discount_pct"] > SAFE_DISCOUNT
]["underwriting_margin"].sum()

high_discount_loss


1449607.9530285296

In [15]:
underpriced_policies = df[
    df["pricing_status"] == "Underpriced"
]

risk_exposure = underpriced_policies["underwriting_margin"].sum()

risk_exposure


0.0

In [17]:
df["profit_flag"] = np.where(
    df["underwriting_margin"] > 0, "Profitable", "Loss"
)
df.head()

,policy_id,policy_start_date,vehicle_type,policy_type,region,vehicle_age,driver_age,base_premium,discount_pct,final_premium,...,underwriting_margin,premium_per_vehicle_age,discount_applied,discount_bucket,avg_vehicle_premium,premium_deviation_pct,risk_adjusted_discount,pricing_status,effective_price_index,profit_flag
0,POL10001,2023-04-13,SUV,Third Party,West,12,41,16600,0,16600.0,...,8924.196385,1276.923077,0,No Discount,16035.203145,3.522231,0.000000,Profitable,1.035222,Profitable
1,POL10002,2023-11-27,Sedan,Third Party,West,7,23,14100,0,14100.0,...,2989.708434,1762.500000,0,No Discount,13138.717949,7.316407,0.000000,Profitable,1.073164,Profitable
2,POL10003,2024-05-06,Luxury,Third Party,West,4,53,19200,0,19200.0,...,7489.870605,3840.000000,0,No Discount,20650.765125,-7.025237,0.000000,Profitable,0.929748,Profitable
3,POL10004,2024-07-14,Sedan,Third Party,West,11,36,13300,0,13300.0,...,7497.973768,1108.333333,0,No Discount,13138.717949,1.227533,0.000000,Profitable,1.012275,Profitable
4,POL10005,2024-07-16,Sedan,Comprehensive,North,2,57,13100,5,12445.0,...,6618.130912,4148.333333,1,Low,13138.717949,-5.279952,1.666667,Profitable,0.947200,Profitable


In [25]:
df["price_sensitivity_zone"] = pd.cut(
    df["effective_price_index"],
    bins=[0, 0.9, 1.0, 1.1, float("inf")],
    labels=[
        "Highly Discounted",
        "Competitive",
        "Market Priced",
        "Overpriced"
    ]
)
df.head()

,policy_id,policy_start_date,vehicle_type,policy_type,region,vehicle_age,driver_age,base_premium,discount_pct,final_premium,...,underwriting_margin,premium_per_vehicle_age,discount_applied,discount_bucket,avg_vehicle_premium,premium_deviation_pct,risk_adjusted_discount,pricing_status,effective_price_index,price_sensitivity_zone
0,POL10001,2023-04-13,SUV,Third Party,West,12,41,16600,0,16600.0,...,8924.196385,1276.923077,0,No Discount,16035.203145,3.522231,0.000000,Profitable,1.035222,Market Priced
1,POL10002,2023-11-27,Sedan,Third Party,West,7,23,14100,0,14100.0,...,2989.708434,1762.500000,0,No Discount,13138.717949,7.316407,0.000000,Profitable,1.073164,Market Priced
2,POL10003,2024-05-06,Luxury,Third Party,West,4,53,19200,0,19200.0,...,7489.870605,3840.000000,0,No Discount,20650.765125,-7.025237,0.000000,Profitable,0.929748,Competitive
3,POL10004,2024-07-14,Sedan,Third Party,West,11,36,13300,0,13300.0,...,7497.973768,1108.333333,0,No Discount,13138.717949,1.227533,0.000000,Profitable,1.012275,Market Priced
4,POL10005,2024-07-16,Sedan,Comprehensive,North,2,57,13100,5,12445.0,...,6618.130912,4148.333333,1,Low,13138.717949,-5.279952,1.666667,Profitable,0.947200,Competitive


In [33]:
np.random.seed(42)

df["claim_cost"] = (
    df["expected_claim_cost"]
    * np.random.normal(1, 0.35, size=len(df))
)


In [35]:
df["underwriting_margin"] = (
    df["final_premium"] - df["claim_cost"]
)


In [37]:
df["profit_flag"] = np.where(
    df["underwriting_margin"] > 0,
    "Profitable",
    "Loss"
)
df.head()

,policy_id,policy_start_date,vehicle_type,policy_type,region,vehicle_age,driver_age,base_premium,discount_pct,final_premium,...,discount_applied,discount_bucket,avg_vehicle_premium,premium_deviation_pct,risk_adjusted_discount,pricing_status,effective_price_index,price_sensitivity_zone,profit_flag,claim_cost
0,POL10001,2023-04-13,SUV,Third Party,West,12,41,16600,0,16600.0,...,0,No Discount,16035.203145,3.522231,0.000000,Profitable,1.035222,Market Priced,Profitable,9010.241717
1,POL10002,2023-11-27,Sedan,Third Party,West,7,23,14100,0,14100.0,...,0,No Discount,13138.717949,7.316407,0.000000,Profitable,1.073164,Market Priced,Profitable,10572.636721
2,POL10003,2024-05-06,Luxury,Third Party,West,4,53,19200,0,19200.0,...,0,No Discount,20650.765125,-7.025237,0.000000,Profitable,0.929748,Competitive,Profitable,14364.710201
3,POL10004,2024-07-14,Sedan,Third Party,West,11,36,13300,0,13300.0,...,0,No Discount,13138.717949,1.227533,0.000000,Profitable,1.012275,Market Priced,Profitable,8894.856944
4,POL10005,2024-07-16,Sedan,Comprehensive,North,2,57,13100,5,12445.0,...,1,Low,13138.717949,-5.279952,1.666667,Profitable,0.947200,Competitive,Profitable,5349.335716


In [39]:
elasticity_profit = (
    df.groupby("price_sensitivity_zone")
      .agg(
          policies=("policy_id", "count"),
          loss_policies=("profit_flag", lambda x: (x == "Loss").sum()),
          avg_margin=("underwriting_margin", "mean")
      )
      .reset_index()
)

elasticity_profit


C:\Users\Abhi\AppData\Local\Temp\ipykernel_15484\1461204291.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("price_sensitivity_zone")


,price_sensitivity_zone,policies,loss_policies,avg_margin
0,Highly Discounted,739,51,4236.594060
1,Competitive,728,42,5323.181702
2,Market Priced,823,46,5938.940453
3,Overpriced,710,48,6184.684774


In [41]:
impact_summary.to_csv(
    "C:/Users/Abhi/Desktop/Python_Data_Analytics_Projects/car_insurance_pricing_analytics/data/processed/discount_impact_summary.csv",
    index=False
)

In [47]:
bucket_impact.to_csv(
    "C:/Users/Abhi/Desktop/Python_Data_Analytics_Projects/car_insurance_pricing_analytics/data/processed/discount_bucket_impact.csv",
    index=False
)

In [49]:
elasticity_profit.to_csv(
    "C:/Users/Abhi/Desktop/Python_Data_Analytics_Projects/car_insurance_pricing_analytics/data/processed/elasticity_profit_summary.csv",
    index=False
)